# Homework 7 Template

Rixin Li & G . Besla



In [ ]:
 # Make edits where instructed - look for "****", which indicates where you need to add code. 

In [11]:
# import necessary modules
# numpy provides powerful multi-dimensional arrays to hold and manipulate data
import numpy as np
# matplotlib provides powerful functions for plotting figures
import matplotlib.pyplot as plt
# astropy provides unit system and constants for astronomical calculations
import astropy.units as u
import astropy.constants as const
# import Latex module so we can display the results with symbols
from IPython.display import Latex
%matplotlib inline

# **** import CenterOfMass to determine the COM pos/vel of M33
import CenterOfMass2 as COM

# **** import the GalaxyMass to determine the mass of M31 for each component
import GalaxyMass as GM

FileNotFoundError: [Errno 2] No such file or directory: 'MW_000.txt'

# M33AnalyticOrbit

In [13]:
class M33AnalyticOrbit:
    """ Calculate the analytical orbit of M33 around M31 """
    
    def __init__(self, filename): # **** add inputs
        """ 
        Initialize the orbit integrator.
        
        Parameters:
        -----------
        filename : str
            Name of the file where the integrated orbit will be saved.
         """

        ### get the gravitational constant (the value is 4.498502151575286e-06)
        self.G = const.G.to(u.kpc**3/u.Msun/u.Gyr**2).value
        
        ### **** store the output file name
        self.fileout = filename
        M33_file = "C:\Users\harni\OneDrive - University of Arizona\Astro400B\Astro400BClone\Homeworks\homework7"
        M31_file = "C:\Users\harni\OneDrive - University of Arizona\Astro400B\Astro400BClone\Homeworks\homework7"
        
        ### get the current pos/vel of M33 
        # **** create an instance of the  CenterOfMass class for M33 
        M33COM = COM.CenterOfMass(M33_file,2)
        # **** store the position VECTOR of the M33 COM (.value to get rid of units)
        self.PM33 = M33COM.COM_P(.1,4).value
        # **** store the velocity VECTOR of the M33 COM (.value to get rid of units)
        self.VM33 = M33COM.COM_V(self.PM33[0]*u.kpc, self.PM33[1]*u.kpc, self.PM33[2]*u.kpc).value
        
        ### get the current pos/vel of M31 
        # **** create an instance of the  CenterOfMass class for M31 
        M31COM = COM.CenterOfMass(M31_file,2)
        # **** store the position VECTOR of the M31 COM (.value to get rid of units)
        self.PM31 = M31COM.COM_P(0.1,2).value
        # **** store the velocity VECTOR of the M31 COM (.value to get rid of units)
        self.VM31 = M31COM.COM_V(self.PM31[0]*u.kpc,self.PM31[1]*u.kpc, self.PM31[2]*u.kpc).value
        
        ### store the DIFFERENCE between the vectors posM33 - posM31
        # **** create two VECTORs self.r0 and self.v0 and have them be the
        # relative position and velocity VECTORS of M33
        self.r = self.PM33 - self.PM31 
        self.v = self.VM33 - self.VM31
        
        ### get the mass of each component in M31 
        ### disk
        # **** self.rdisk = scale length (no units)
        self.rdisk = 5
        # **** self.Mdisk set with ComponentMass function. Remember to *1e12 to get the right units. Use the right ptype
        self.Mdisk = GM.ComponentMass(M31_file, 2) * 1e12
        ### bulge
        # **** self.rbulge = set scale length (no units)
        self.rbulge = 1
        # **** self.Mbulge  set with ComponentMass function. Remember to *1e12 to get the right units Use the right ptype
        self.Mbulge = GM.ComponentMass(M31_file,  3) * 1e12 
        # Halo
        # **** self.rhalo = set scale length from HW5 (no units)
        self.rhalo = 60
        # **** self.Mhalo set with ComponentMass function. Remember to *1e12 to get the right units. Use the right ptype
        self.Mhalo = GM.ComponentMass(M31_file, 1) * 1e12
    
    
    def HernquistAccel(self, M, r_a, r): # it is easiest if you take as an input the position VECTOR 
        """ 
        Calculates the Halo and Bulge acceleration using the Hernquist profile.
        inputs:
            M = Mass of halo or bulge
            r_a = the scale length of the halo or bulge (default == self.rhalo or self.rbulge)
            r = position (default == self.r)
        Output:
            Hern = acceleration vector of the halo or bulge
        """
        
        ### **** Store the magnitude of the position vector
        rmag = np.sqrt(np.sum(r**2))
        
        ### *** Store the Acceleration
        Hern = -(self.G*M)*r/(rmag*(r_a + rmag)**2)#follow the formula in the HW instructions
        # NOTE: we want an acceleration VECTOR so you need to make sure that in the Hernquist equation you 
        # use  -G*M/(rmag *(ra + rmag)**2) * r --> where the last r is a VECTOR 
        
        return Hern
    
    
    
    def MiyamotoNagaiAccel(self, M, r_d, r):# it is easiest if you take as an input a position VECTOR  r 
        """
        Calculates the disk acceleration
        inputs:
            M = Mass of disk
            r_d = the scale length of the disk (default == self.rdisk)
            r = position (default == self.r)
        """

        
        ### Acceleration **** follow the formula in the HW instructions
        R = np.sqrt(r[0]**2 + r[1]**2) # this is the distance in the x-y plane
        z_d=r_d/5.0
        B = r_d + np.sqrt(r[2]**2 + z_d**2) # this is the distance in the z direction
        MNa = -(self.G*M)*r/(R**2 + B**2)**(3/2) * np.array([1,1, B/np.sqrt(r[2]**2+z_d**2)]) # this is the acceleration in the x-y plane
        # AGAIN note that we want a VECTOR to be returned  (see Hernquist instructions)
        # this can be tricky given that the z component is different than in the x or y directions. 
        # we can deal with this by multiplying the whole thing by an extra array that accounts for the 
        # differences in the z direction:
        #  multiply the whle thing by :   np.array([1,1,ZSTUFF]) 
        # where ZSTUFF are the terms associated with the z direction
        
        
       
        return MNa
        # the np.array allows for a different value for the z component of the acceleration
     
    
    def M31Accel(self, r): # input should include the position vector, r
        """
        This is the function that will sum all of the acceleration vectors from each galaxy component
        Input:
            r = position vector in 3D space (x,y,z)
        Output:
            3D acceleration vector of M31
        """

        ### Call the previous functions for the halo, bulge and disk
        # **** these functions will take as inputs variable we defined in the initialization of the class like 
        # self.rdisk etc. 
        Halo_acc = self.HernquistAccel(self.Mhalo, self.rhalo, r) # halo acceleration
        Bulge_acc = self.HernquistAccel(self.Mbulge, self.rbulge, r) # bulge acceleration
        Disk_acc = self.MiyamotoNagaiAccel(self.Mdisk, self.rdisk, r) # disk acceleration    
            # return the SUM of the output of the acceleration functions - this will return a VECTOR 
        total_acc = np.sum([Halo_acc, Bulge_acc, Disk_acc], axis=0) # sum the three acceleration vectors
        return total_acc
    
    
    
    def LeapFrog(self, dt, r, v): # take as input r and v, which are VECTORS. Assume it is ONE vector at a time
        """ **** ADD COMMENTS """
        
        # predict the position at the next half timestep
        rhalf = r + v * dt / 2
        a = self.M31Accel(rhalf)  # get the acceleration at the half timestep position
        
        # predict the final velocity at the next timestep using the acceleration field at the rhalf position 
        vnew = v + a * dt
        
        # predict the final position using the average of the current velocity and the final velocity
        # this accounts for the fact that we don't know how the speed changes from the current timestep to the 
        # next, so we approximate it using the average expected speed over the time interval dt. 
        rnew = rhalf + vnew * dt / 2
        
        # return the new position and velocity vectors
        return rnew, vnew
    
    
    
    def OrbitIntegration(self, t0, dt, tmax):
         """
         Function that loops over the LeapFrog integrator to solve the equation of motion and compute future orbits
         Input:
                t0 : float
                    Initial time (Gyr)
                dt : float
                    Time step (Gyr)
                tmax : float
                    Maximum time (Gyr)
        Output:
                orbit : ndarray
                    Array containing the time, position, and velocity at each time step.
         """

        # initialize the time to the input starting time
        t = t0
        
        # initialize an empty array of size :  rows int(tmax/dt)+2  , columns 7
        orbit = np.zeros((int(tmax/dt)+2, 7))
        
        # initialize the first row of the orbit
        orbit[0] = t0, *tuple(self.r), *tuple(self.v)
        # this above is equivalent to 
        # orbit[0] = t0, self.r0[0], self.r0[1], self.r0[2], self.v0[0], self.v0[1], self.v0[2]
        
        
        # initialize a counter for the orbit.  
        i = 1 # since we already set the 0th values, we start the counter at 1
        
        # start the integration (advancing in time steps and computing LeapFrog at each step)
        while (t < tmax):  # as long as t has not exceeded the maximal time 
            
            # **** advance the time by one timestep, dt
            t = t+dt
            # **** store the new time in the first column of the ith row
            orbit[i,0] = t
            r_old = orbit[i-1, 1:4] # get the old position vector from the previous row
            v_old = orbit[i-1, 4:7] # get the old velocity vector from the previous row
            # ***** advance the position and velocity using the LeapFrog scheme
            rnew, vnew = self.LeapFrog(dt, r_old, v_old) # get the new position and velocity vectors
            # remember that LeapFrog returns a position vector and a velocity vector  
            # as an example, if a function returns three vectors you would call the function and store 
            # the variable like:     a,b,c = function(input)
            
         
    
            # ****  store the new position vector into the columns with indexes 1,2,3 of the ith row of orbit
            orbit[i, 1:4] = rnew[0], rnew[1], rnew[2]
            # TIP:  if you want columns 5-7 of the Nth row of an array called A, you would write : 
            # A[n, 5:8] 
            # where the syntax is row n, start at column 5 and end BEFORE column 8
            
            
            # ****  store the new position vector into the columns with indexes 1,2,3 of the ith row of orbit
            orbit[i, 4:7]= vnew[0], vnew[1], vnew[2]
            
            # **** update counter i , where i is keeping track of the number of rows (i.e. the number of time steps)
            i += 1
        print(self.fileout)
        
        # write the data to a file
        np.savetxt("C:\Users\harni\OneDrive - University of Arizona\Astro400B\Astro400BClone\Homeworks\homework7"+self.filename, orbit, fmt = "%11.3f"*7, comments='#', 
                   header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                   .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))
        
        # there is no return function
        
        

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 170)